In [9]:
#Daily Feature
import pandas as pd
import numpy as np
import datetime as datetime
import pytse_client as tse
from pytse_client import download_client_types_records

In [10]:
tickers = tse.download(symbols="فولاد", adjust=True, write_to_csv=True)
df = pd.DataFrame.from_dict(tickers["فولاد"])
df.head()

,date,open,high,low,adjClose,value,volume,count,yesterday,close
0,2007-03-11,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0,16.0
1,2007-03-12,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0,16.0
2,2007-03-13,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0,16.0
3,2007-03-14,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0,16.0
4,2007-03-17,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0,16.0


In [11]:
if __name__ == '__main__':
    records_dict = download_client_types_records("فولاد")
    df1 = pd.DataFrame.from_dict(records_dict["فولاد"])


df1.reindex(list(range(0, 3912))).reset_index(drop=True)

#Rearrange from past to present with reindex
df1 = df1.iloc[::-1]
df1.index = np.arange(len(df1))
df1.head()

,date,individual_buy_count,corporate_buy_count,individual_sell_count,corporate_sell_count,individual_buy_vol,corporate_buy_vol,individual_sell_vol,corporate_sell_vol,individual_buy_value,corporate_buy_value,individual_sell_value,corporate_sell_value,individual_buy_mean_price,individual_sell_mean_price,corporate_buy_mean_price,corporate_sell_mean_price,individual_ownership_change
0,2008-11-26,14,2,4,1,106338,20010,126338,10,248937258,46844120,295757258,24120,2341.000000,2341.0,2341.035482,2412.000000,-20000.0
1,2008-11-29,116,175,3,2,4718258,33691903,121402,38288759,14980229312,107713006031,284080680,122409154663,3174.949168,2340.0,3196.999767,3196.999795,4596856.0
2,2008-11-30,10,2,1,1,87219,12791,100000,10,204005241,29918859,233900000,24100,2339.000000,2339.0,2339.055508,2410.000000,-12781.0
3,2008-12-01,8,1,5,1,128666,10,128666,10,300821108,24090,300821108,24090,2338.000000,2338.0,2409.000000,2409.000000,0.0
4,2008-12-02,7,1,1,0,18553,10000,28553,0,43358361,23370000,66728361,0,2337.000000,2337.0,2337.000000,NaN,-10000.0


In [12]:
def candel_based_feature(df):
    print('weekly candles features')

    def candle_based(df):
        df.index = [pd.Timestamp(item) for item in df['date']]
        return_df = pd.DataFrame({\
            'max_price7d': df['high'].rolling(5).max(),\
            'min_price7d': df['low'].rolling(5).min(),\
            'first_price7d': df['open'].rolling(5).agg(lambda rows: rows[0]),\
            'last_price7d': df['close'].rolling(5).agg(lambda rows: rows[-1]),\
            'yesterday_price7d': df['yesterday'].rolling(5).agg(lambda rows: rows[0]),\
            'max_price14d': df['high'].rolling(10).max(),\
            'min_price14d': df['low'].rolling(10).min(),\
            'first_price14d': df['open'].rolling(10).agg(lambda rows: rows[0]),\
            'last_price14d': df['close'].rolling(10).agg(lambda rows: rows[-1]),\
            'yesterday_price14d': df['yesterday'].rolling(10).agg(lambda rows: rows[0]),\
            'max_price21d': df['high'].rolling(15).max(),\
            'min_price21d': df['low'].rolling(15).min(),\
            'first_price21d': df['open'].rolling(15).agg(lambda rows: rows[0]),\
            'last_price21d': df['close'].rolling(15).agg(lambda rows: rows[-1]),\
            'yesterday_price21d': df['yesterday'].rolling(15).agg(lambda rows: rows[0]),\
            'max_price30d': df['high'].rolling(23).max(),\
            'min_price30d': df['low'].rolling(23).min(),\
            'first_price30d': df['open'].rolling(23).agg(lambda rows: rows[0]),\
            'last_price30d': df['close'].rolling(23).agg(lambda rows: rows[-1]),\
            'yesterday_price30d': df['yesterday'].rolling(23).agg(lambda rows: rows[0]),\
            'max_price60d': df['high'].rolling(44).max(),\
            'min_price60d': df['low'].rolling(44).min(),\
            'first_price60d': df['open'].rolling(44).agg(lambda rows: rows[0]),\
            'last_price60d': df['close'].rolling(44).agg(lambda rows: rows[-1]),\
            'yesterday_price60d': df['yesterday'].rolling(44).agg(lambda rows: rows[0]),\
            'max_price90d': df['high'].rolling(70).max(),\
            'min_price90d': df['low'].rolling(70).min(),\
            'first_price90d': df['open'].rolling(70).agg(lambda rows: rows[0]),\
            'last_price90d': df['close'].rolling(70).agg(lambda rows: rows[-1]),\
            'yesterday_price90d': df['yesterday'].rolling(70).agg(lambda rows: rows[0]),\
            })
        return return_df
    df_candle_based = candle_based(df)


    def shadow_based(df):
        return_df = pd.DataFrame({\
            'shadow_up7d': (df['max_price7d'] - np.maximum(df['first_price7d'],df['last_price7d']))/
                                (df['yesterday_price7d']),\
            'shadow_low7d': (df['min_price7d'] - np.minimum(df['first_price7d'],df['last_price7d']))/
                                (df['yesterday_price7d']),\
            'body7d': np.abs(df['last_price7d']-df['first_price7d'])/df['yesterday_price7d'],\
            'shadow_up14d': (df['max_price14d'] - np.maximum(df['first_price14d'],df['last_price14d']))/
                                (df['yesterday_price14d']),\
            'shadow_low14d': (df['min_price14d'] - np.minimum(df['first_price14d'],df['last_price14d']))/
                                (df['yesterday_price14d']),\
            'body14d': np.abs(df['last_price14d']-df['first_price14d'])/df['yesterday_price14d'],\
            'shadow_up21d': (df['max_price21d'] - np.maximum(df['first_price21d'],df['last_price21d']))/
                                (df['yesterday_price21d']),\
            'shadow_low21d': (df['min_price21d'] - np.minimum(df['first_price21d'],df['last_price21d']))/
                                (df['yesterday_price21d']),\
            'body21d': np.abs(df['last_price21d']-df['first_price21d'])/df['yesterday_price21d'],\
            'shadow_up30d': (df['max_price30d'] - np.maximum(df['first_price30d'],df['last_price30d']))/
                                (df['yesterday_price30d']),\
            'shadow_low30d': (df['min_price30d'] - np.minimum(df['first_price30d'],df['last_price30d']))/
                                (df['yesterday_price30d']),\
            'body30d': np.abs(df['last_price30d']-df['first_price30d'])/df['yesterday_price30d'],\
            'shadow_up60d': (df['max_price60d'] - np.maximum(df['first_price60d'],df['last_price60d']))/
                                (df['yesterday_price60d']),\
            'shadow_low60d': (df['min_price60d'] - np.minimum(df['first_price60d'],df['last_price60d']))/
                                (df['yesterday_price60d']),\
            'body60d': np.abs(df['last_price60d']-df['first_price60d'])/df['yesterday_price60d'],\
            'shadow_up90d': (df['max_price90d'] - np.maximum(df['first_price90d'],df['last_price90d']))/
                                (df['yesterday_price90d']),\
            'shadow_low90d': (df['min_price90d'] - np.minimum(df['first_price90d'],df['last_price90d']))/
                                (df['yesterday_price90d']),\
            'body90d': np.abs(df['last_price90d']-df['first_price90d'])/df['yesterday_price90d']\
            })
        return return_df
    df_shadow_based = candle_based(df)


    df_feature = pd.concat([df_shadow_based, df_candle_based], axis=1)
    return df_feature
    
    
candel_based_feature(df)

weekly candles features


,max_price7d,min_price7d,first_price7d,last_price7d,yesterday_price7d,max_price14d,min_price14d,first_price14d,last_price14d,yesterday_price14d,...,max_price60d,min_price60d,first_price60d,last_price60d,yesterday_price60d,max_price90d,min_price90d,first_price90d,last_price90d,yesterday_price90d
2007-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-17,16.0,16.0,16.0,16.0,16.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-03,5350.0,5020.0,5350.0,5080.0,5330.0,5450.0,5020.0,5420.0,5080.0,5560.0,...,5850.0,4926.0,5465.0,5080.0,5493.0,6047.0,4926.0,5545.0,5080.0,5545.0
2022-09-04,5280.0,5020.0,5270.0,5090.0,5270.0,5420.0,5020.0,5420.0,5090.0,5420.0,...,5850.0,4926.0,5470.0,5090.0,5446.0,6047.0,4926.0,5714.0,5090.0,5653.0
2022-09-05,5280.0,5020.0,5200.0,5060.0,5230.0,5380.0,5020.0,5230.0,5060.0,5310.0,...,5850.0,4926.0,5437.0,5060.0,5446.0,6047.0,4926.0,5601.0,5060.0,5601.0
2022-09-06,5190.0,5020.0,5160.0,5070.0,5170.0,5380.0,5020.0,5330.0,5070.0,5270.0,...,5850.0,4926.0,5432.0,5070.0,5451.0,6047.0,4926.0,5578.0,5070.0,5578.0
